In [126]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier as RF


### Toy Data

In [127]:
# Don't change the ranomd seed or n
np.random.seed(0)
n = 250
smart_factor = np.random.normal(0, 15, n)
smart_factor_norm = smart_factor / np.max(smart_factor)
iq = 100 + smart_factor
iq = iq.astype(int)
education = np.round(14 + np.random.randint(-4, 2, n) + smart_factor_norm * 5).astype(int)
female = np.random.randint(0, 2, n).astype(bool)
age = 30 + np.random.randint(-10, 20, n)
income_level = np.random.choice(['low', 'medium', 'high'], n, p=[0.4, 0.4, 0.2])
criminal_val = -.15 *iq + -2.5*education + -5*female + -15*(income_level == 'high') + -10*(income_level == 'medium') + -10 * (age > 28) + np.random.randint(-20, 20, n)

# take the bottom 10% of the criminal values and label them as criminals
criminals = criminal_val > np.percentile(criminal_val, 90)

# construct a pandas dataframe with the data
df = pd.DataFrame({'iq': iq, 'education': education, 'female': female, 'age': age, 'income_level': income_level, 'criminals': criminals})
df.loc[3, 'criminals'] = True
df.loc[246, 'criminals'] = True
crime_data = df.__deepcopy__()

#### Helper Functions

In [128]:
# Find a function that returns the number of rows in a given array
def num_rows(array):
    """ Returns the number of rows in a given array """
    if array is None:
        return 0
    elif len(array.shape) == 1:
        return 1
    else:
        return array.shape[0]

# Define a function that gives number of samples under each class label
def class_counts(data):
    """ Returns a dictionary with the number of samples under each class label
        formatted {label : number_of_samples} """
    if len(data.shape) == 1: # If there's only one row
        return {data[-1] : 1}
    counts = {}
    for label in data[:,-1]:
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

# Define a function that calculates the Gini impurity of a given array
def info_gain(data, left, right):
    """Return the info gain of a partition of data.
    Parameters:
        data (ndarray): the unsplit data
        left (ndarray): left split of data
        right (ndarray): right split of data
    Returns:
        (float): info gain of the data"""
        
    def gini(data):
        """Return the Gini impurity of given array of data.
        Parameters:
            data (ndarray): data to examine
        Returns:
            (float): Gini impurity of the data"""
        counts = class_counts(data)
        N = num_rows(data)
        impurity = 1
        for lbl in counts:
            prob_of_lbl = counts[lbl] / N
            impurity -= prob_of_lbl**2
        return impurity
        
    p = num_rows(right)/(num_rows(left)+num_rows(right))
    return gini(data) - p*gini(right)-(1-p)*gini(left)



############## WORK HERE ##############
def find_best_split(data, tree_type, size_rand_subset, min_samples_leaf, is_numeric):
    # Get feature_choice, define refinement, and initialize the best gain and current_bool
    feature_choice = np.random.choice(data.shape[1], size_rand_subset, replace=False)
    refinement = 100/np.min([1+np.abs(data.shape[0] -2*min_samples_leaf -2)// min_samples_leaf,40])
    current_bool = None
    current_gain = 0

    # Initialize the best gain and question and loop through the first n-1 columns
    for i in feature_choice:
        screen = data[:,i]

        # if it is numeric, get the unique split values based on the refinement
        if is_numeric[i]:
            split_vals = np.unique(np.percentile(screen, np.arange(refinement, 100, refinement)))

            # Loop through the split values and partition the data
            for value in split_vals:
                bool_splits = (screen >= value) 

                # If the partition is too small, skip it
                if (np.sum(bool_splits) < min_samples_leaf) or (np.sum(~bool_splits) < min_samples_leaf):
                    continue

                # Calculate the info gain and update the best gain and question if necessary
                gain = info_gain(data, data[bool_splits], data[~bool_splits])
                if gain > current_gain:
                    current_gain, current_bool = gain, bool_splits
        
        # If it is not numeric, get the unique values and the size
        else:
            categories = np.unique(screen)
            cat_size = len(categories)

            # If there are only two categories, there is only one possible split
            if cat_size == 2:
                cat_splits = [[0]]

            # If there are more than two categories, there are many different possible splits. Split individually and then choose random splits
            else:
                cat_splits = [i for i in range(cat_size)]
                random_splits = [np.random.choice(categories, size, replace=False) for size in np.arange(2, cat_size-1, 1)]
                if len(random_splits) > 0:
                    cat_splits.append(random_splits)
                
            # Get the unique splits
            for split in cat_splits:
                bool_splits = np.isin(screen, split)

                # If the partition is too small, skip it
                if (np.sum(bool_splits) < min_samples_leaf) or (np.sum(~bool_splits) < min_samples_leaf):
                    continue
            
                # Calculate the info gain and update the best gain and question if necessary
                gain = info_gain(data, data[bool_splits], data[~bool_splits])
                if gain > current_gain:
                    current_gain, current_bool = gain, bool_splits

    # Return the best gain and split
    return current_gain, current_bool


# Wilson's Code
def find_best_split(data, tree_type, size_rand_subset, min_samples_leaf, is_numeric):
    """Find the optimal split
    Parameters:
        data (ndarray): Data in question
        feature_names (list of strings): Labels for each column of data
        min_samples_leaf (int): minimum number of samples per leaf
        random_subset (bool): for Problem 6
    Returns:
        (float): Best info gain
        (Question): Best question"""
    # Initialize variables
    best_gain = 0
    best_question = None

    m, n = data.shape
    n -= 1
    feature_indices = np.arange(n)

    if random_subset:  # If it is random
        num_feat = int(np.sqrt(n))
        feature_indices = np.random.choice(feature_indices, num_feat, replace=False)

    # Iterate through each feature (column) (Do not iterate through final column
    for j in feature_indices:
        # Iterate through each unique value (row)
        for i in range(m):
            # Create Question obj with column and value
            question = Question(j, data[i, j], feature_names=feature_names)

            # Use partition to split the dataset into left and right partitions
            left, right = partition(data, question)

            # If either left or right partitions are smaller than allowable leaf size reject
            if num_rows(left) < min_samples_leaf:
                continue
            elif num_rows(right) < min_samples_leaf:
                continue

            # Calculate Info Gain
            gain = info_gain(data, left, right)

            # Update best_gain and best_question
            if gain > best_gain:
                best_gain = gain
                best_question = question

    return best_gain, best_question


In [129]:
#### REFERENCE ####
def predict_tree(sample, my_tree):
    """Predict the label for a sample given a pre-made decision tree
    Parameters:
        sample (ndarray): a single sample
        my_tree (Decision_Node or Leaf): a decision tree
    Returns:
        Label to be assigned to new sample"""

    # Base case if my_tree is a leaf
    if isinstance(my_tree, Leaf):
        return max(my_tree.prediction, key=my_tree.prediction.get)

    # Otherwise break it down into left and right branches
    if my_tree.question.match(sample):
        return predict_tree(sample, my_tree.left)
    else:
        return predict_tree(sample, my_tree.right)


def analyze_tree(dataset, my_tree):
    """Test how accurately a tree classifies a dataset
    Parameters:
        dataset (ndarray): Labeled data with the labels in the last column
        tree (Decision_Node or Leaf): a decision tree
    Returns:
        (float): Proportion of dataset classified correctly"""

    # Get labels
    actual_labels = dataset[:, -1]
    n = dataset.shape[0]

    # Predict each sample in dataset
    predicted_labels = np.array(
        [predict_tree(dataset[i], my_tree) for i in range(n)])

    return (actual_labels == predicted_labels).sum() / n


def predict_forest(sample, forest):
    """Predict the label for a new sample, given a random forest
    Parameters:
        sample (ndarray): a single sample
        forest (list): a list of decision trees
    Returns:
        Label to be assigned to new sample"""
    n = len(forest)

    # Predict the sample label using each tree
    predicted_labels = np.array(
        [predict_tree(sample, forest[i]) for i in range(n)]).astype(int)

    return np.argmax(np.bincount(predicted_labels))


def analyze_forest(dataset, forest):
    """Test how accurately a forest classifies a dataset
    Parameters:
        dataset (ndarray): Labeled data with the labels in the last column
        forest (list): list of decision trees
    Returns:
        (float): Proportion of dataset classified correctly"""

    # Get labels
    actual_labels = dataset[:, -1]

    n = dataset.shape[0]

    # Predict each sample in the dataset
    predicted_labels = np.array(
        [predict_forest(dataset[i], forest) for i in range(n)])

    return (actual_labels == predicted_labels).sum() / n



#### Helper Classes

In [130]:

class Question:
    """Questions to use in construction and display of Decision Trees.
    Attributes:
        column (int): which column of the data this question asks
        value (int/float): value the question asks about
        features (str): name of the feature asked about
    Methods:
        match: returns boolean of if a given sample answered T/F"""

    def __init__(self, column, value, feature_names):
        # Store attributes
        self.column = column
        self.value = value
        self.features = feature_names[self.column]

    def match(self, sample):
        """Returns T/F depending on how the sample answers the question
        Parameters:
            sample ((n,), ndarray): New sample to classify
        Returns:
            (bool): How the sample compares to the question"""
        # Check if it is a match
        return sample[self.column] >= self.value


class Leaf:
    """Tree leaf node
    Attribute:
        prediction (dict): Dictionary of labels at the leaf"""
    def __init__(self, data):
        # Store attributes
        self.prediction = class_counts(data)
        self.train_size = num_rows(data)
    
    # Define a method to get the train size
    def get_train_size(self):
        return self.train_size

    # Define a method to update the test size
    def update_test_size(self, data):
        self.test_size = num_rows(data)

    # Define a method to get the test size
    def get_test_size(self):
        return self.test_size


class Decision_Node:
    """Tree node with a question
    Attributes:
        question (Question): Question associated with node
        left (Decision_Node or Leaf): child branch
        right (Decision_Node or Leaf): child branch"""
    def __init__(self, question, left_branch, right_branch):
        # Store attributes
        self.question = question
        self.left = left_branch
        self.right = right_branch


### Main Classes (DecisionTree, AdaptiveForest)

In [131]:
class DecisionTree:
    """
    A decision tree class that can be used for classification.
    Attributes:
        data (numpy array): data to use for the tree
        features (list): list of feature names
        min_samples_leaf (int): minimum number of samples in a leaf
        max_depth (int): maximum depth of the tree
        is_numeric (numpy array - boolean): boolean array indicating whether each feature is numeric or not
        size_random_subset (int): number of features to use for each split
        y (numpy array): target variable
        X (numpy array): data without the target variable
    """

    def __init__(self, data, tree_type, features, min_samples_leaf, max_depth, is_numeric, size_random_subset):
        """
        Initialize the decision tree
        Inputs:
            data (numpy array): data to use for the tree (target in last column)
            tree_type (str): type of tree (classification or regression)
            features (list): list of feature names
            min_samples_leaf (int): minimum number of samples in a leaf
            max_depth (int): maximum depth of the tree
            is_numeric (numpy array - boolean): boolean array indicating whether each feature is numeric or not
            size_random_subset (int): number of features to use for each split
        """
        # Initialize the different attributes of the tree
        self.data = data
        self.tree_type = tree_type
        self.features = features
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth
        self.is_numeric = is_numeric
        self.size_random_subset = size_random_subset

    # Define a method to build a tree
    def build_tree(self, data, current_depth=0):
        """Build a classification tree using the classes Decision_Node and Leaf
        Parameters:
            data (ndarray)
            min_samples_leaf (int): minimum allowed number of samples per leaf
            max_depth (int): maximum allowed depth
            current_depth (int): depth counter
            random_subset (bool): whether or not to train on a random subset of features
        Returns:
            Decision_Node (or Leaf)"""
        # If the number of rows is less than the minimum samples per leaf, return a leaf
        if data.shape[0] < 2*self.min_samples_leaf:
            return Leaf(data)
        
        # Find the best question to ask and return a leaf if there is no gain or past max depth
        gain, question = find_best_split(data, self.tree_type, self.size_random_subset, self.min_samples_leaf, self.is_numeric)
        if gain == 0 or current_depth >= self.max_depth:
            return Leaf(data)
        
        # Partition the data and build the left and right branches
        bool_vals = question.match(data)
        left, right = data[bool_vals], data[~bool_vals]
        left_branch = self.build_tree(left, current_depth+1)
        right_branch = self.build_tree(right, current_depth+1)

        # Return a Decision_Node with the best question and branches
        return Decision_Node(question, left_branch, right_branch)
    
    # Define a method to fit the tree
    def fit(self):
        self.tree = self.build_tree(self.data)


class AdaptiveForest:
    """
    A random forest class that can be used for classification.
    Attributes:
        data (numpy array): data to use for the tree
        features (list): list of feature names
        min_samples_leaf (int): minimum number of samples in a leaf
        max_depth (int): maximum depth of the tree
        is_numeric (numpy array - boolean): boolean array indicating whether each feature is numeric or not
        size_random_subset (int): number of features to use for each split
        y_index (int): index of the column of the target variable
    """
    
    def __init__(self,dataframe, labels = None, target_index = None, n_trees = 10, min_samples_leaf=5, max_depth=4, features = None, size_rand_subset = None, bootstrap_size = None):
        """
        Initialize the random forest
        Inputs:
            dataframe (pandas dataframe or numpy array): data to use for the forest
            target_index (int): index of the column of the target variable
            min_samples_leaf (int): minimum number of samples in a leaf
            max_depth (int): maximum depth of the tree
            features (list): list of feature names
            is_numeric (numpy array - boolean): boolean array indicating whether each feature is numeric or not
            category_codes (dict): dictionary of the category codes for each non numeric feature
            size_random_subset (int): number of features to use for each split
        """
        #### Data preprocessing ####
        # Check the data type and raise an error if it is not a dataframe or numpy array
        if isinstance(dataframe, np.ndarray):
            df = pd.DataFrame(dataframe)

            # Check if the data is numeric and convert it if it is not
            def infer_dtype(col):
                try:
                    return pd.to_numeric(col, errors='raise')
                except:
                    return col
            df = df.apply(lambda col: infer_dtype(col))
            if features is not None:
                df.columns = features
        
        # If the data is a dataframe, then make a copy of it, raise an error otherwise
        elif isinstance(dataframe, pd.DataFrame):
            df = dataframe.__deepcopy__()
        else:
            raise ValueError('Data must be a pandas dataframe or numpy array')

        # If labels are given, then make them a series and concatenate them to the dataframe if conditions met
        if labels is not None:
            labels = pd.Series(labels)
            if target_index is not None:
                raise ValueError('Cannot have both labels and target index')
            
            # check if labels and dataframe have the same number of rows
            if len(labels) != len(df):
                raise ValueError('Labels and dataframe must be same size')
            
            # concatenate the labels to the dataframe
            df = pd.concat([df, labels], axis=1)

        # If labels are not given, then check if target index is given and move the target variable to the last column
        else:
            if target_index is None:
                print('Warning: No target index given. Using last index as target variable.')
            else:
                cols = list(df.columns)
                cols.append(cols.pop(target_index))
                df = df[cols]

        #### Initialize tree attributes ####
        # Save the features except for the target variable and check the target column data type
        self.features = np.array(df.columns)
        target_dtype = df[self.features[-1]].dtype

        # check if the target variable is numeric
        if isinstance(target_dtype, (int, float, complex)):
            self.forest_type = 'regression'
        else:
            self.forest_type = 'classification'

        ######## TO DO: add regression functionality ########
        if self.forest_type == 'regression':
            raise ValueError('Regression not yet implemented')
        
        #####################################################

        # Check which features are numeric and save the boolean mask
        numeric_cols = df.select_dtypes(include=['number']).columns
        self.is_numeric = df.columns.isin(numeric_cols)
        self.category_codes = {}
     
        # Loop through non numeric columns and convert them to numeric while saving the category codes
        for col in self.features[~self.is_numeric]:
            self.category_codes[col] = dict(enumerate(df[col].astype('category').cat.categories))
            df[col] = df[col].astype('category').cat.codes
    
        # Save the size of the random subset for attribute bagging
        if size_rand_subset is None:
            self.size_rand_subset = int(np.ceil(np.sqrt(len(self.features) - 1)))
        else:
            self.size_rand_subset = size_rand_subset

        # Initialize remaining tree attributes
        self.data = df.values.astype(float)
        self.min_samples_leaf = min_samples_leaf
        self.max_depth = max_depth

        #### Other forest attributes ####
        if bootstrap_size is None:
            self.bootstrap_size = df.shape[0]
        else:
            self.bootstrap_size = bootstrap_size
        self.n_trees = n_trees
        self.weights = np.ones(self.n_trees) / self.n_trees


    def access_tree(self):
        """ Access the decision tree class """
        return DecisionTree(self.data, self.forest_type, self.features, self.min_samples_leaf, self.max_depth, self.is_numeric, self.size_rand_subset)
        

    ######## FIX ME ########
    def fit(self, test_size = .33):
        """ Fit the random forest """
        # Initialize the list of trees
        self.trees = []
        #self.oob_data = []

        # Loop through the number of trees and fit each tree to bootstrapped data
        for i in range(self.n_trees):
            # boot strap our data
            n_samples = len(self.data)
            indices = np.random.randint(0, n_samples, size=n_samples)
            boot_data = self.data[indices]

            # get the out of bag data
            # oob_indices = np.setdiff1d(np.arange(n_samples), indices, assume_unique=True)
            # oob_data = self.data[oob_indices]
            # self.oob_data.append(oob_data)

            # Build the tree and fit it
            tree = DecisionTree(boot_data, self.features, self.min_samples_leaf, self.max_depth, self.is_numeric, self.size_rand_subset)
            tree.fit()

            # Save the tree and the normalized weights
            self.trees.append(tree)


In [133]:
crime_data1 = crime_data.copy()
crime_data1[crime_data1.columns[0]] += .1
tester = AdaptiveForest(crime_data1)
tree = tester.access_tree()

tree.fit()



NameError: name 'random_subset' is not defined